In [ ]:
using Knet, Plots, Images
gr()

atype   = Array{Float32}; # atype = KnetArray{Float32} for gpu usage, Array{Float32} for cpu. 
nepochs = 10
fast    = false

println("OS: ", Sys.KERNEL)
println("Julia: ", VERSION)
println("Knet: ", Pkg.installed("Knet"))


### New Stuff

In this notebook we introduce the following Julia/Knet packages and functions:

* Knet's function [conv4](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.conv4): Execute convolutions or cross-correlations using filters specified with `w` over tensor `x`.
* Knet's function [pool](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.pool): Compute pooling of input values (i.e., the maximum or average of several adjacent values) to produce an output with smaller height and/or width.

## Convolutional neural networks (CNNs)

In the [previous example](../chapter03_deep-neural-networks/section2-multilayer perceptrons.ipynb), we connected the nodes of our neural networks in what seems like the simplest possible way. Every node in each layer was connected to every node in the subsequent layers. 

![](https://github.com/zackchase/mxnet-the-straight-dope/blob/master/img/multilayer-perceptron.png?raw=true)

This can require a lot of parameters! If our input were a 256x256 color image (still quite small for a photograph), and our network had 1,000 nodes in the first hidden layer, then our first weight matrix would require (256x256x3)x1000 parameters. That's nearly 200 million. Moreover the hidden layer would ignore all the spatial structure in the input image even though we know the local structure represents a powerful source of prior knowledge. 

Convolutional neural networks incorporate convolutional layers. These layers associate each of their nodes with a small window, called a *receptive field*, in the previous layer, instead of connecting to the full layer. This allows us to first learn local features via transformations that are applied in the same way for the top right corner as for the bottom left. Then we collect all this local information to predict global qualities of the image (like whether or not it depicts a dog). 

![](http://cs231n.github.io/assets/cnn/depthcol.jpeg)
(Image credit: Stanford cs231n http://cs231n.github.io/assets/cnn/depthcol.jpeg)

In short, there are two new concepts you need to grok here. First, we'll be introducting *convolutional* layers. Second, we'll be interleaving them with *pooling* layers. 

In [ ]:
# could do it Sebastian's way....

for p in ("GZip",)
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using GZip

"Where to download mnist from"
mnisturl = "http://yann.lecun.com/exdb/mnist"

"Where to download mnist to"
mnistdir = "./"

"""
This utility loads the [MNIST](http://yann.lecun.com/exdb/mnist)
hand-written digits dataset.  There are 60000 training and 10000 test
examples. Each input x consists of 784 pixels representing a 28x28
grayscale image.  The pixel values are converted to Float32 and
normalized to [0,1].  Each output y is a UInt8 indicating the correct
class.  10 is used to represent the digit 0.
```
# Usage:
include(Pkg.dir("Knet/data/mnist.jl"))
xtrn, ytrn, xtst, ytst = mnist()
# xtrn: 28×28×1×60000 Array{Float32,4}
# ytrn: 60000-element Array{UInt8,1}
# xtst: 28×28×1×10000 Array{Float32,4}
# ytst: 10000-element Array{UInt8,1}
```
"""
function mnist()
    global _mnist_xtrn,_mnist_ytrn,_mnist_xtst,_mnist_ytst
    if !isdefined(:_mnist_xtrn)
        info("Loading MNIST...")
        _mnist_xtrn = _mnist_xdata("train-images-idx3-ubyte.gz")
        _mnist_xtst = _mnist_xdata("t10k-images-idx3-ubyte.gz")
        _mnist_ytrn = _mnist_ydata("train-labels-idx1-ubyte.gz")
        _mnist_ytst = _mnist_ydata("t10k-labels-idx1-ubyte.gz")
    end
    return _mnist_xtrn,_mnist_ytrn,_mnist_xtst,_mnist_ytst
end

"Utility to view a MNIST image, requires the Images package"
mnistview(x,i)=colorview(Gray,permutedims(x[:,:,1,i],(2,1)))

function _mnist_xdata(file)
    a = _mnist_gzload(file)[17:end]
    reshape(a ./ 255f0, (28,28,1,div(length(a),784)))
end

function _mnist_ydata(file)
    a = _mnist_gzload(file)[9:end]
    a[a.==0] = 10
    # full(sparse(a,1:length(a),1f0,10,length(a)))
    return a
end

function _mnist_gzload(file)
    if !isdir(mnistdir)
        mkpath(mnistdir)
    end
    path = joinpath(mnistdir,file)
    if !isfile(path)
        url = "$mnisturl/$file"
        download(url, path)
    end
    f = gzopen(path)
    a = read(f)
    close(f)
    return(a)
end
xtrn, ytrn, xtst, ytst = mnist()


In [ ]:
num_samples = 1000;
batch_size  = 10;
dtrn = minibatch(xtrn[:, :, :, 1:num_samples], ytrn[1:num_samples], batch_size, xtype=atype);
dtst = minibatch(xtst[:, :, :, 1:num_samples], ytst[1:num_samples], batch_size, xtype=atype);

##  Parameters

Each node in convolutional layer is associated with a 3D block (`height` x `width` x `channel`) in the input tensor. Moreover, the convolutional layer itself has multiple output channels. So the layer is parameterized by a 4 dimensional weight tensor, commonly called a *convolutional kernel*.

The output is produced by sliding the kernel across the input image skipping locations according to a pre-defined *stride* (but we'll just assume that to be 1 in this tutorial). Let's initialize some such kernels from scratch.

In [ ]:
# (kx, ky, input_size, output_size)
initweights(atype) = map(atype, Any[ xavier(Float32,5,5,1,20),  zeros(Float32,1,1,20,1),
                                     xavier(Float32,5,5,20,50), zeros(Float32,1,1,50,1),
                                     xavier(Float32,500,800),   zeros(Float32,500,1),
                                     xavier(Float32,10,500),    zeros(Float32,10,1) ]);

The shape of each kernel is ($k_x, k_y$, ch_input, ch_output), where $k_x, k_y$ are the kernel size (sliding window size), ch_input is the number of input channels (1 for gray scale, 3 for RGB), and ch_output is the number of output channels (or output filters). Basically, the number of output channels is really the number of filters we want to create. Each new filter is convolved with each input filter. For the first layer the input is the image $x$, therefore we convolve 20 filters with a single input image (for RGB, we convolved all three images). 

## Model 

We will build our `predict` function by introducing two new functions. First, we will new Knet's function [conv4](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.conv4) to execute convolutions or cross-correlations using filters specified with `w` over tensor `x`. After that we will use Knet's function [pool](http://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.pool). Pooling gives us a way to downsample in the spatial dimensions. Early convnets typically used average pooling, but max pooling tends to give better results. 

![](https://qph.ec.quoracdn.net/main-qimg-8afedfb2f82f279781bfefa269bc6a90)

A typical pooling layer is defined by the `window size` (typically 2x2), `stride size` (typically (2,2)), and `pooling type`. 



In [ ]:
#

function predict(w,x0)
    x1 = pool(relu.(conv4(w[1],x0) .+ w[2]))
    x2 = pool(relu.(conv4(w[3],x1) .+ w[4]))
    x3 = relu.(w[5]*mat(x2) .+ w[6])
    return w[7]*x3 .+ w[8]
end


Recall that in the first model from the last lecture's notebook ("**Binary classification with logistic regression**") our loss function was the *log loss*; in the second model ("**Multiclass Logistic Regression 
**") our loss function was the generalization of log loss to more than two classes, known as *cross-entropy*.

For this convolutional model, since we are again predicting from among 10 classes, we will use a cross-entropy loss function taken from the Knet library, [Knet.nll](https://denizyuret.github.io/Knet.jl/latest/reference.html#Knet.nll).

In [ ]:
loss(w,x,ygold, predict) = nll(predict(w,x), ygold);
lossgradient = grad(loss)

## Training procedure

For a default of 10 epochs (i.e., loops through the entire training dataset `dtrn`), update the weights based on the gradient computed from the loss function.

In [ ]:
function train(w, dtrn, optim, predict; epochs=10)
    for epoch = 1:epochs
        for (x, y) in dtrn
            g = lossgradient(w,x,y,predict)
            update!(w,g,optim)
        end
    end
end

## Optimizer (SGD) and reporting

In [ ]:
optim(w; lr=0.1) = optimizers(w, Sgd;  lr=lr);

In [ ]:
function report(epoch, w, dtrn, dtst, predict)
    println((:epoch, epoch, :trn, accuracy(w, dtrn, predict), :tst, accuracy(w, dtst, predict)))
end

## Train the Model

In [ ]:
w = initweights(atype);
opt = optim(w);

for epoch = 1:nepochs
    train(w, dtrn, opt, predict; epochs=1)
    report(epoch, w, dtrn, dtst, predict)
end


Not bad! consider that we only used 1000 samples, and this time around we were able to achieve 93% accuracy on the `test` dataset. 

## Examining Predictions

In [ ]:
testdata = collect(enumerate(dtst))
myinstance = testdata[12][2]
tstimg = myinstance[1]
tstval = myinstance[2]

println("tstval is " , convert(UInt32, tstval[1]))
mnistview(tstimg,1)


## Conclusion

Contained in this example are nearly all the important ideas you'll need to start attacking problems in computer vision. While state-of-the-art vision systems incorporate a few more bells and whistles, they're all built on this foundation. Believe it or not, if you knew just the content in this tutorial 5 years ago, you could probably have sold a startup to a Fortune 500 company for millions of dollars. Fortunately (or unfortunately?), the world has gotten marginally more sophisticated, so we'll have to come up with some more sophisticated tutorials to follow.